<a href="https://colab.research.google.com/github/Emdey/RSA-encryption-SYSTEM/blob/main/RSA_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Create Configuration Directory**

This ensures the folder structure exists for your custom theme file.

In [22]:
!mkdir -p .streamlit

**Configure Custom Color Theme**

This sets the dark blue primary accent and navy sidebar color as requested.

In [23]:
%%writefile .streamlit/config.toml
[theme]
# Dark Blue Primary color for buttons, links, and accents
primaryColor="#000080"
# White background
backgroundColor="#FFFFFF"
# Deep Navy Blue for sidebar and input area backgrounds
secondaryBackgroundColor="#ADD8E6"
# Dark text color
textColor="#000000"
font="sans serif"

Overwriting .streamlit/config.toml


**Create the Customized Application Script (rsa_app.py)**

This cell contains your original RSA mathematical functions and Streamlit interface, with only the title and page icon changed.

In [24]:
%%writefile rsa_app.py
import streamlit as st
import random
import math
import time

# --- 1. CORE RSA FUNCTIONS (Original math functions preserved) ---
def gcd(a, b):
    while b: a, b = b, a % b
    return a
def modInverse(a, m):
    m0, x0, x1 = m, 0, 1
    if m == 1: return 0
    while a > 1:
        q = a // m
        m, a = a % m, m
        x0, x1 = x1 - q * x0, x0
    if x1 < 0: x1 += m0
    return x1

def miller_rabin_test(n, k=5):
    if n <= 3: return n > 1
    if n % 2 == 0: return False
    r, s = 0, n - 1
    while s % 2 == 0: r += 1; s //= 2
    for _ in range(k):
        a = random.randrange(2, n - 1)
        x = pow(a, s, n)
        if x == 1 or x == n - 1: continue
        for _ in range(r - 1):
            x = pow(x, 2, n)
            if x == n - 1: break
        else: return False
    return True

def generate_prime(n):
    while True:
        p = random.getrandbits(n)
        if p > 1 and p % 2 != 0 and miller_rabin_test(p): return p

def generate_keypair(p, q):
    n = p * q
    phi = (p - 1) * (q - 1)
    e = 65537
    d = modInverse(e, phi)
    return ((e, n), (d, n))

def encrypt(pk, plaintext):
    e, n = pk
    ciphertext = [pow(ord(char), e, n) for char in plaintext]
    return ciphertext

def decrypt(pk, ciphertext):
    d, n = pk
    decrypted_chars = [chr(pow(char, d, n)) for char in ciphertext]
    return ''.join(decrypted_chars)

# --- 2. CUSTOM STREAMLIT WEB INTERFACE (Title and Icon Changed) ---
st.set_page_config(page_title="Secure RSA Vault", page_icon="🛡️", layout="wide")
st.title("🛡️ RSA Vault: Cryptography")
st.markdown("---")

# **Key Generation Section - UPDATED TO SELECTBOX**
st.sidebar.subheader("Key Size Selection")
modulus_size = st.sidebar.selectbox(
    "Select Final RSA Key Size (Modulus n):",
    options=[64, 128, 512, 1024, 2048, 4096],
    index=3, # Default to 1024
    key='modulus_size_select'
)

# Calculate the required prime size (p and q are half the modulus size)
key_size = modulus_size // 2

st.sidebar.caption("⚠️ WARNING: Generating keys > 128 bits with custom Python math is very slow and may time out in Colab.")

# FIX: Separate button and logic for robust state management
generate_button = st.sidebar.button("🔑 Generate New Keys", type="primary", use_container_width=True)

if generate_button or 'public_key' not in st.session_state:
    st.session_state.force_key_gen = True

if st.session_state.get('force_key_gen'):
    # Aggressively clear old keys
    if 'public_key' in st.session_state: del st.session_state.public_key
    if 'private_key' in st.session_state: del st.session_state.private_key
    if 'keys' in st.session_state: del st.session_state.keys

    with st.spinner(f'Generating {key_size}-bit keys... This may take a moment.'):
        st.session_state.p = generate_prime(key_size)
        st.session_state.q = generate_prime(key_size)

        public_key, private_key = generate_keypair(st.session_state.p, st.session_state.q)

        st.session_state.public_key = public_key
        st.session_state.private_key = private_key

    # Reset flag so generation doesn't repeat unnecessarily
    st.session_state.force_key_gen = False

    # Set a flag to force the sidebar to re-render entirely
    st.session_state.sidebar_render_count = st.session_state.get('sidebar_render_count', 0) + 1


# Ensure keys exist before accessing them
if 'public_key' in st.session_state:
    pub_key = st.session_state.public_key
    priv_key = st.session_state.private_key

    st.sidebar.subheader("Current Keys")
    st.sidebar.markdown(f"**Public Key (e, n):** `{pub_key}`")
    st.sidebar.markdown(f"Key Size: {key_size * 2}-bit (Final Modulus)") # Display the actual modulus size

    # --- EASY ACCESS FOR PRIVATE KEY COMPONENTS (RETAINED) ---
    st.sidebar.markdown("---")
    st.sidebar.subheader("🔒 Private Key Components")

    # Exponent (d)
    n_val = priv_key[1]
    d_val = priv_key[0]

    st.sidebar.text_area(
        "Private Key Exponent (d) - Copy this value:",
        value=str(d_val),
        height=70,
        key=f"d_sidebar_{n_val}"
    )

    # Modulus (n)
    st.sidebar.text_area(
        "Modulus (n) - Copy this value:",
        value=str(n_val),
        height=70,
        key=f"n_sidebar_{n_val}"
    )
    # --- END RETAINED FEATURE ---


    # **Main Application Area: Local Encrypt/Decrypt (RETAINED)**
    st.header("1. Local Encryption & Decryption")
    col1, col2 = st.columns(2)

    with col1:
        st.subheader("Message Input")
        message_input = st.text_area(
            "Enter your message here:",
            value="This is my RSA deployment!",
            height=150
        )

        # RETAINED: Dropdown for Encrypt Only vs. Full Cycle
        operation = st.selectbox(
            "Select Operation:",
            ("Full Encrypt & Decrypt Cycle", "Only Encrypt")
        )

        if st.button("🚀 **Run Local Operation**", use_container_width=True, type="primary"):
            st.session_state.run_local_app = True
        else:
            if 'run_local_app' not in st.session_state:
                st.session_state.run_local_app = False

    with col2:
        st.subheader("Output")
        if st.session_state.run_local_app:

            with st.spinner('Encrypting...'):
                ciphertext = encrypt(pub_key, message_input)
                time.sleep(1)

            st.success(f"**Ciphertext (List of Integers):**")
            st.code(ciphertext)

            if operation == "Full Encrypt & Decrypt Cycle":
                st.markdown("---")
                with st.spinner('Decrypting...'):
                    decrypted_message = decrypt(priv_key, ciphertext)
                    time.sleep(1)

                st.info(f"**Decrypted Message:**")
                st.code(decrypted_message)

                if decrypted_message == message_input:
                    st.balloons()
                    st.success("✅ Decryption Successful: Original message recovered!")
                else:
                    st.error("❌ Decryption Failed.")

            st.session_state.run_local_app = False

else:
    st.warning("Please click 'Generate New Keys' in the sidebar to begin.")


# --- SECTION: EXTERNAL DECRYPTION (RETAINED) ---
st.markdown("---")
st.header("2. Decrypt External Ciphertext (Supports Large Keys)")
st.caption("Use this to decrypt ciphertext encrypted by someone else's public key, provided you have the corresponding private key (d, n).")

with st.form("external_decrypt_form"):
    # Input for Ciphertext
    cipher_input = st.text_area(
        "Enter Ciphertext (list of integers, e.g., [123, 456, 789]):",
        height=100
    )

    # RETAINED: Input for Private Key components d and n
    d_input = st.text_input(
        "Enter Private Key Exponent (d):",
        value="",
        placeholder="Copy this value from the Private Key Components section above."
    )
    n_input = st.text_input(
        "Enter Modulus (n):",
        value="",
        placeholder="Copy this value from the Private Key Components section above."
    )

    decrypt_button = st.form_submit_button("🔓 **Decrypt External Message**", type="primary")

if decrypt_button:
    if not cipher_input or not d_input or not n_input:
        st.error("Please enter a valid ciphertext, Exponent (d), and Modulus (n).")
    else:
        try:
            # Safely convert the string input to a list of integers
            external_ciphertext = eval(cipher_input)
            if not isinstance(external_ciphertext, list) or not all(isinstance(i, int) for i in external_ciphertext):
                raise ValueError("Ciphertext must be a list of integers.")

            # Convert string inputs to Python large integers
            d_val = int(d_input)
            n_val = int(n_input)

            private_key_external = (d_val, n_val)

            with st.spinner('Decrypting External Message...'):
                decrypted_message_external = decrypt(private_key_external, external_ciphertext)
                time.sleep(2)

            st.success("✅ Decryption Complete!")
            st.info(f"**Decrypted Message:**")
            st.code(decrypted_message_external)

        except (ValueError, SyntaxError) as e:
            st.error(f"Input Error: Could not process ciphertext or keys. Ensure ciphertext is a correct list of integers and keys are valid large integers. Details: {e}")
        except Exception as e:
            st.error(f"A decryption error occurred: {e}. Check that the private key (d, n) is correct.")

Overwriting rsa_app.py


**Install Required Libraries & Cleanup**

This cell installs the necessary libraries (pyngrok and streamlit) and performs the essential cleanup before the final launch.

In [25]:
!pip install -q streamlit pyngrok

import os
from pyngrok import ngrok
import time

# --- STEP 1: SET NGROK TOKEN (Use the token from your original file) ---
NGROK_AUTH_TOKEN = "34rp0vyiJkGRukoDBTWN4cd2r6c_5BQvHRDoHWzuRZfjPRsLm"
try:
    ngrok.set_auth_token(NGROK_AUTH_TOKEN)
except Exception as e:
    print(f"Error setting ngrok token: {e}")
    raise

# --- STEP 2: AGGRESSIVE CLEANUP ---
print("\nCleaning up old processes...")
# Kill Streamlit process
os.system("pkill streamlit")
# Kill ngrok process
os.system("pkill ngrok")
time.sleep(5)


Cleaning up old processes...


In [26]:
!pip install -q streamlit pyngrok pycryptodome

In [27]:
import os
from pyngrok import ngrok
import time

# 1. Kill the Streamlit server process
print("Cleaning up old Streamlit processes...")
os.system("pkill streamlit")
time.sleep(2) # Give the process time to shut down

# 2. Kill all existing ngrok tunnels
print("Killing all existing ngrok tunnels...")
ngrok.kill()

print("Cleanup complete. You can now re-run the deployment cell.")

Cleaning up old Streamlit processes...
Killing all existing ngrok tunnels...
Cleanup complete. You can now re-run the deployment cell.


**Deploy the Application**

This final cell launches the server and creates the public tunnel.

In [28]:
# 1. Start Streamlit in the background (&)
print("Starting Secure RSA Vault on port 8501...")
os.system("streamlit run rsa_app.py &")

# 2. Wait for the server to initialize
time.sleep(15)

# 3. Create the public ngrok tunnel
print("Creating public ngrok tunnel...")
try:
    public_url = ngrok.connect(addr="8501", proto="http")

    print("\n" + "="*80)
    print("🛡️ Your RSA App is Live and Deployed!")
    print(f"🔗 Public URL: {public_url}")
    print("="*80 + "\n")

except Exception as e:
    print(f"Failed to create ngrok tunnel: {e}")
    print("\n---")
    print("⚠️ Deployment Failed. Please check your ngrok token and ensure the cleanup cell was run successfully.")

Starting Secure RSA Vault on port 8501...
Creating public ngrok tunnel...

🛡️ Your RSA App is Live and Deployed!
🔗 Public URL: NgrokTunnel: "https://nutty-earnestine-homogenetic.ngrok-free.dev" -> "http://localhost:8501"

